In [8]:
import numpy as np
from scipy.optimize import minimize, differential_evolution, dual_annealing

def interp_matrix(theta_vals):
    # create interpolation matrix
    return np.array([[1/np.sqrt(2), np.cos(val), np.sin(val)] for val in theta_vals])

def objective(theta_vals, x):
    A = interp_matrix(theta_vals)
    Var = np.linalg.inv(A.T @ A)
    part = Var[1:3, 1:3]  # Fixed to select the correct submatrix
    v = np.array([-np.sin(x), np.cos(x)])
    return np.dot(np.dot(v.T, part), v)  # Fixed the order of np.dot for correct matrix multiplication

def objective2(theta_vals, x):
    A = interp_matrix(theta_vals)
    Var = np.linalg.inv(A.T)
    t = np.array([0, -np.sin(x), np.cos(x)])
    At = np.dot(Var, t)
    return np.dot(At, At)  # Fixed the order of np.dot for correct matrix multiplication


# Example usage
# np.random.seed(4)
# x = np.random.uniform(0, 2 * np.pi, size=1)[0]

x = 0

# Define the bounds for theta_vals
bounds = [(0, 2 * np.pi) for _ in range(3)]


# Global optimization using differential evolution
result_de = differential_evolution(objective2, bounds, args=(x,))
print("Global Optimization Method: Differential Evolution")
print("Optimized theta_vals:", result_de.x)
print("Minimum value of the objective function:", result_de.fun)
print("Optimization success:", result_de.success)
print("Optimization message:", result_de.message)
print()

# Global optimization using differential evolution
result_de = dual_annealing(objective2, bounds, args=(x,))
print("Global Optimization Method: dual_annealing")
print("Optimized theta_vals:", result_de.x)
print("Minimum value of the objective function:", result_de.fun)
print("Optimization success:", result_de.success)
print("Optimization message:", result_de.message)
print()

Global Optimization Method: Differential Evolution
Optimized theta_vals: [4.71139451 1.57128432 1.57229727]
Minimum value of the objective function: 0.3750004189699107
Optimization success: True
Optimization message: Optimization terminated successfully.

Global Optimization Method: dual_annealing
Optimized theta_vals: [4.71326568 1.5706239  4.71185751]
Minimum value of the objective function: 0.37500011337160555
Optimization success: True
Optimization message: ['Maximum number of iteration reached']



In [9]:
import numpy as np
from scipy.optimize import minimize, differential_evolution, dual_annealing

def interp_matrix(theta_vals, max_n):
    # 创建插值矩阵，每行依次包含 1, cos(θ), sin(θ), cos(2θ), sin(2θ), ..., cos(nθ), sin(nθ)
    return np.array([[1/np.sqrt(2)] + [func(k * val) for k in range(1, max_n + 1) for func in (np.cos, np.sin)] for val in theta_vals])

def define_vector_t(x, max_n):
    vector = [0]  # 初始化向量，包含第一个元素0
    for i in range(1, max_n+1):
        vector.append(-i * np.sin(i * x))
        vector.append(i * np.cos(i * x))
    return np.array(vector)

def objective1(theta_vals, x, max_n):
    A = interp_matrix(theta_vals, max_n)
    Var = np.linalg.inv(A.T @ A)
    t = define_vector_t(x, max_n)
    return np.dot(t.T, np.dot(Var, t))  # Fixed the order of np.dot for correct matrix multiplication

def objective2(theta_vals, x, max_n):
    A = interp_matrix(theta_vals, max_n)
    invAT = np.linalg.inv(A.T)
    t = define_vector_t(x, max_n)
    invATt = np.dot(invAT, t)
    return np.dot(invATt, invATt)  # Fixed the order of np.dot for correct matrix multiplication

def vandermonde_matrix(x, column_major=False):
    n = len(x)
    V = np.zeros((n, n), dtype=complex)
    for i in range(n):
        for j in range(n):
            V[i, j] = x[j] ** i
    return V.T if column_major else V

def objective3(theta_vals, x, max_n):
    w = [np.exp(1j * theta) for theta in theta_vals]
    V = vandermonde_matrix(w, column_major=True)
    y = np.array([k * np.exp(-1j * k * x) for k in range(max_n, 0, -1)] + [0] + [-k * np.exp(1j * k * x) for k in range(1, max_n + 1)])

    invV = np.linalg.inv(V)
    yinvV = y.conj().T @ invV
    return np.linalg.norm(yinvV, 2) ** 2  # Fixed the order of np.dot for correct matrix multiplication

max_n = 4
x = 0

theta_vals = np.random.uniform(0, 2 * np.pi, 2 * max_n + 1)

print(objective1(theta_vals, x, max_n))
print(objective2(theta_vals, x, max_n))
print(objective3(theta_vals, x, max_n))


886.6961646765872
886.6961646765961
886.6961646765985


In [12]:
import numpy as np
from scipy.optimize import minimize, differential_evolution, dual_annealing

# 定义你之前的函数...
# ...

# 设置固定的 x 和 max_n 值
max_n = 3
x = 0

# 随机初始化 theta_vals
theta_vals_init = np.random.uniform(0, 2 * np.pi, 2 * max_n + 1)

# 定义优化的边界
bounds = [(0, 2 * np.pi) for _ in range(2 * max_n + 1)]

# 使用 `minimize` 求解（BFGS 方法）
result_minimize_obj1 = minimize(objective1, theta_vals_init, args=(x, max_n), method='BFGS')
result_minimize_obj2 = minimize(objective2, theta_vals_init, args=(x, max_n), method='BFGS')
result_minimize_obj3 = minimize(objective3, theta_vals_init, args=(x, max_n), method='BFGS')

print("Minimize (BFGS) result for objective1:", result_minimize_obj1.fun)
print("Minimize (BFGS) result for objective2:", result_minimize_obj2.fun)
print("Minimize (BFGS) result for objective3:", result_minimize_obj3.fun)

# # 使用 `differential_evolution` 求解
# result_de_obj1 = differential_evolution(objective1, bounds, args=(x, max_n))
# result_de_obj2 = differential_evolution(objective2, bounds, args=(x, max_n))
# result_de_obj3 = differential_evolution(objective3, bounds, args=(x, max_n))

# print("Differential Evolution result for objective1:", result_de_obj1.fun)
# print("Differential Evolution result for objective2:", result_de_obj2.fun)
# print("Differential Evolution result for objective3:", result_de_obj3.fun)

# # 使用 `dual_annealing` 求解
# result_da_obj1 = dual_annealing(objective1, bounds, args=(x, max_n))
# result_da_obj2 = dual_annealing(objective2, bounds, args=(x, max_n))
# result_da_obj3 = dual_annealing(objective3, bounds, args=(x, max_n))

# print("Dual Annealing result for objective1:", result_da_obj1.fun)
# print("Dual Annealing result for objective2:", result_da_obj2.fun)
# print("Dual Annealing result for objective3:", result_da_obj3.fun)


Minimize (BFGS) result for objective1: 2.1183898216158923
Minimize (BFGS) result for objective2: 2.118327758775108
Minimize (BFGS) result for objective3: 2.1183286308852516


In [151]:

# Define the bounds for theta_vals
bounds = [(0, 2 * np.pi) for _ in range(2 * max_n + 1)]

# Global optimization using differential evolution
result_de = differential_evolution(objective1, bounds, args=(x,max_n,))
print("Global Optimization Method: Differential Evolution")
print("Optimized theta_vals:", result_de.x)
print("Minimum value of the objective function:", result_de.fun)
print("Optimization success:", result_de.success)
print("Optimization message:", result_de.message)
print()

# Global optimization using differential evolution
result_de = dual_annealing(objective2, bounds, args=(x,max_n,))
print("Global Optimization Method: dual_annealing")
print("Optimized theta_vals:", result_de.x)
print("Minimum value of the objective function:", result_de.fun)
print("Optimization success:", result_de.success)
print("Optimization message:", result_de.message)
print()

# Global optimization using differential evolution
result_de = dual_annealing(objective3, bounds, args=(x, max_n,))
print("Global Optimization Method: dual_annealing")
print("Optimized theta_vals:", result_de.x)
print("Minimum value of the objective function:", result_de.fun)
print("Optimization success:", result_de.success)
print("Optimization message:", result_de.message)
print()


LinAlgError: Singular matrix

In [ ]:
import numpy as np
from scipy.optimize import differential_evolution, dual_annealing
import matplotlib.pyplot as plt

def interp_matrix(theta_vals, max_n):
    # 创建插值矩阵，每行依次包含 1, cos(θ), sin(θ), cos(2θ), sin(2θ), ..., cos(nθ), sin(nθ)
    return np.array([[1/np.sqrt(2)] + [func(k * val) for k in range(1, max_n + 1) for func in (np.cos, np.sin)] for val in theta_vals])

def define_vector_t(x, max_n):
    vector = [0]  # 初始化向量，包含第一个元素0
    for i in range(1, max_n+1):
        vector.append(-i * np.sin(i * x))
        vector.append(i * np.cos(i * x))
    return np.array(vector)

def objective1(theta_vals, x, max_n):
    A = interp_matrix(theta_vals, max_n)
    Var = np.linalg.inv(A.T @ A)
    t = define_vector_t(x, max_n)
    return np.dot(t.T, np.dot(Var, t))

def objective2(theta_vals, x, max_n):
    A = interp_matrix(theta_vals, max_n)
    invAT = np.linalg.inv(A.T)
    t = define_vector_t(x, max_n)
    invATt = np.dot(invAT, t)
    return np.dot(invATt, invATt)

def safe_optimize(optimizer, objective, bounds, args, max_retries=100):
    for attempt in range(max_retries):
        try:
            result = optimizer(objective, bounds, args=args)
            return result.fun
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}. Retrying...")
    raise RuntimeError(f"Optimization failed after {max_retries} attempts.")

def optimize_and_compare(max_n_values, x):
    differential_evolution_results = []
    dual_annealing_results = []
    theoretical_values = []
    
    for max_n in max_n_values:
        bounds = [(0, 2 * np.pi) for _ in range(2 * max_n + 1)]
        
        # Optimization with differential evolution for objective1
        de_result = safe_optimize(differential_evolution, objective1, bounds, args=(x, max_n,))
        differential_evolution_results.append(de_result)
        
        # # Optimization with dual annealing for objective2
        # da_result = safe_optimize(dual_annealing, objective2, bounds, args=(x, max_n,))
        # dual_annealing_results.append(da_result)
        
        # Theoretical value
        r = max_n
        theoretical_value = (2 * r**2 + 1) / 6
        theoretical_values.append(theoretical_value)
        
    
    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(max_n_values, differential_evolution_results, label='Differential Evolution Result')
    # plt.plot(max_n_values, dual_annealing_results, label='Dual Annealing Result')
    plt.plot(max_n_values, theoretical_values, label=r'Theoretical $\frac{2r^2 + 1}{6}$', linestyle='--')
    plt.xlabel('max_n')
    plt.ylabel('Objective Function Value')
    plt.title('Comparison of Optimization Results and Theoretical Value')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(differential_evolution_results)   
    print(dual_annealing_results)
    print(theoretical_values) 

# Example usage
max_n_values = range(1, 11)  # Test for max_n from 1 to 10
x = np.pi/2  # You can change this value if needed
optimize_and_compare(max_n_values, x)
